# Model Selection Claim Approval

Selecting a model that performs as the best classifier for predicting whether an ePA will be approved or not.

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

sns.set_theme(style="whitegrid")
sns.set(rc={'figure.figsize':(8.0,6.0)})

In [24]:
data = pd.read_csv('data/training/train.csv')
test = pd.read_csv('data/training/test.csv')

In [10]:
# data.head()
# data.profile_report()

In [25]:
# We focus on the claims that were not approved, and require an ePA.
data = data[data['pharmacy_claim_approved']==0]
test = test[test['pharmacy_claim_approved']==0]

In [14]:
# data.head()
# data.profile_report()

We can drop the id's, since these are useful indices but not useful for classification.

In [26]:
id_columns = ['dim_pa_id','dim_date_id','dim_claim_id','Unnamed: 0']

We can also drop the year, since this is not cyclical.

In [27]:
date_columns = ['calendar_year','date_val']

In [28]:
data = data.drop(columns=id_columns+date_columns+['pharmacy_claim_approved'])
test = test.drop(columns=id_columns+date_columns+['pharmacy_claim_approved'])

In [19]:
data.head()

,calendar_month,calendar_day,day_of_week,is_weekday,is_workday,is_holiday,bin,drug,reject_code,pharmacy_claim_approved,correct_diagnosis,tried_and_failed,contraindication,pa_approved
2,11,11,2,1,1,0,417740,B,70,0,1.0,1.0,0.0,0.0
3,6,28,6,1,1,0,999001,A,76,0,1.0,0.0,0.0,1.0
11,7,31,4,1,1,0,417740,C,75,0,0.0,0.0,0.0,1.0
19,2,21,4,1,1,0,417614,B,75,0,1.0,1.0,0.0,1.0
20,6,28,5,1,1,0,417740,C,75,0,0.0,1.0,0.0,0.0


In [29]:
def encode(data,feature,prefix=''):
    for feat in np.unique(data[feature]):
        data[prefix+'_'+str(feat)] = pd.get_dummies(data[feature])[feat]
    data = data.drop(columns=[feature])
    return data

In [30]:
# Encode the data 

data = encode(data,'drug','drug')

test = encode(test,'drug','drug')

data = encode(data,'bin','payer')

test = encode(test,'bin','payer')

data = encode(data,'reject_code','reject_code')

test = encode(test,'reject_code','reject_code')

In [31]:
data.head()

,calendar_month,calendar_day,day_of_week,is_weekday,is_workday,is_holiday,correct_diagnosis,tried_and_failed,contraindication,pa_approved,drug_A,drug_B,drug_C,payer_417380,payer_417614,payer_417740,payer_999001,reject_code_70,reject_code_75,reject_code_76
2,11,11,2,1,1,0,1.0,1.0,0.0,0.0,0,1,0,0,0,1,0,1,0,0
3,6,28,6,1,1,0,1.0,0.0,0.0,1.0,1,0,0,0,0,0,1,0,0,1
11,7,31,4,1,1,0,0.0,0.0,0.0,1.0,0,0,1,0,0,1,0,0,1,0
19,2,21,4,1,1,0,1.0,1.0,0.0,1.0,0,1,0,0,1,0,0,0,1,0
20,6,28,5,1,1,0,0.0,1.0,0.0,0.0,0,0,1,0,0,1,0,0,1,0


In [68]:
X = data.drop(columns=['pa_approved'])
y = data['pa_approved']

X_test = test.drop(columns=['pa_approved'])
y_test = test['pa_approved']

## Models to Assess 

We will compare the following models 
* baseline: classify all ePA's as approved
* logistic regression
* random forest
* decision tree
* extra trees
* Naive Bayes classifier 
* linear support vector machine
* adaptive boosting 
* gradient boosting
* xg boosting 

In [69]:
# Import our models 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.ensemble import GradientBoostingClassifier


from xgboost import XGBClassifier

In [70]:
# Import some useful metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

In [71]:
# Define our baseline "classifier" 

def ePA_base_classifier(X):
    
    return np.array([1]*X.shape[0])

In [72]:
metrics = {'accuracy':accuracy_score,'precision':precision_score,'recall':recall_score,
          'f1':f1_score,'roc_auc':roc_auc_score}

In [73]:
from sklearn.model_selection import StratifiedKFold, KFold

In [85]:
def evaluate_model(model):
    
    scores = {metric:[] for metric in metrics.keys()}
    SKF = StratifiedKFold(n_splits=5)
    k = 0
    for train_idx, test_idx in SKF.split(X.values, y.values):
        print('On data split # ',k)
        model.fit(X.values[train_idx],y.values[train_idx])
        ykpred = model.predict(X.values[test_idx])
        for i, (metric,score) in enumerate(metrics.items()):
            scores[metric].append(score(y.values[test_idx], ykpred))
        k+=1
    
    return pd.DataFrame(scores)

In [90]:
### Evaluate the models

In [87]:
DT_scores = evaluate_model(DecisionTreeClassifier())

On data split #  0
On data split #  1
On data split #  2
On data split #  3
On data split #  4


In [89]:
DT_scores.mean()

accuracy     0.784465
precision    0.845472
recall       0.864675
f1           0.854965
roc_auc      0.713503
dtype: float64

In [91]:
RF_scores = evaluate_model(RandomForestClassifier())

On data split #  0
On data split #  1
On data split #  2
On data split #  3
On data split #  4


In [92]:
RF_scores.mean()

accuracy     0.791577
precision    0.838005
recall       0.887970
f1           0.862264
roc_auc      0.706298
dtype: float64

In [93]:
ET_scores = evaluate_model(ExtraTreesClassifier())

On data split #  0
On data split #  1
On data split #  2
On data split #  3
On data split #  4


In [94]:
ET_scores.mean()

accuracy     0.787493
precision    0.845711
recall       0.869364
f1           0.857374
roc_auc      0.715063
dtype: float64

In [98]:
LR_scores = evaluate_model(LogisticRegression(penalty='l2', C=1))

On data split #  0


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On data split #  1


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On data split #  2


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On data split #  3


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


On data split #  4


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [99]:
LR_scores.mean()

accuracy     0.814436
precision    0.838770
recall       0.925307
f1           0.879911
roc_auc      0.716350
dtype: float64

In [100]:
LSV_scores = evaluate_model(LinearSVC())

On data split #  0


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


On data split #  1


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


On data split #  2


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


On data split #  3


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


On data split #  4


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [101]:
LSV_scores.mean()

accuracy     0.810401
precision    0.855662
recall       0.894737
f1           0.873667
roc_auc      0.735790
dtype: float64

In [102]:
ADB_scores = evaluate_model(AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
            n_estimators = 10,
            algorithm="SAMME.R",
            learning_rate = 0.5))

On data split #  0
On data split #  1
On data split #  2
On data split #  3
On data split #  4


In [103]:
ADB_scores.mean()

accuracy     0.800597
precision    0.814769
recall       0.942971
f1           0.874195
roc_auc      0.674640
dtype: float64

In [104]:
NB_scores = evaluate_model(CategoricalNB())

On data split #  0
On data split #  1
On data split #  2
On data split #  3
On data split #  4


In [105]:
NB_scores.mean()

accuracy     0.736345
precision    0.928585
recall       0.694561
f1           0.794699
roc_auc      0.773312
dtype: float64

In [106]:
GB_scores = evaluate_model(GradientBoostingClassifier())

On data split #  0
On data split #  1
On data split #  2
On data split #  3
On data split #  4


In [108]:
XG_scores = evaluate_model(XGBClassifier())

On data split #  0


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:21:26] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
On data split #  1
[10:21:34] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


On data split #  2
[10:21:43] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


On data split #  3


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:21:53] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
On data split #  4
[10:22:02] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/Emily/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [110]:
XG_scores.mean()

accuracy     0.813478
precision    0.843860
recall       0.915529
f1           0.878234
roc_auc      0.723194
dtype: float64

### Hyper Parameter Tuning

In [116]:
NB_scores = evaluate_model(CategoricalNB())

On data split #  0
On data split #  1
On data split #  2
On data split #  3
On data split #  4


In [117]:
NB_scores.mean()

accuracy     0.736345
precision    0.928585
recall       0.694561
f1           0.794699
roc_auc      0.773312
dtype: float64

In [120]:
NB_scores = evaluate_model(CategoricalNB(alpha=0.1))

On data split #  0
On data split #  1
On data split #  2
On data split #  3
On data split #  4


In [121]:
NB_scores.mean()

accuracy     0.736345
precision    0.928585
recall       0.694561
f1           0.794699
roc_auc      0.773312
dtype: float64

In [122]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
rf_random.best_params_